In [3]:
import ee, os
from pathlib import Path
from datetime import datetime
from tqdm import tqdm
import time
import rasterio
import rasterio.warp
from utils import get_project_data

In [4]:
# Project data
gt_file, gt, gt_date, gt_crs, polygon, bounds = get_project_data(
    "498",
    gt_dir="../assets/data/preprocessed",
    gt_data_bands=list(range(1,6)),
    shapefile_paths=['../assets/data/NHM_projectDekning_AOI_edit2015_V2.shp', 
                    '../assets/data/ALS_projects_Dz_all_norway.shp']
)

In [7]:
bounds[0][0]

[(9.703842793092072, 60.7253957927164),
 (10.258106843349504, 60.933251765941094)]

In [5]:
import ee
ee.Initialize(project="ee-walteliot")

In [131]:
import geetools
import ee
from uuid import uuid4
from geetools.utils import makeName
from geetools import tools
from geetools.batch import utils
from hashlib import sha256

def exportImageCollectionToDrive(
    collection, 
    fn_prefix, 
    folder, 
    scale, 
    datatype, 
    region, 
    crs, 
    verbose=False, 
    **kwargs
):
    """
    Slight modification of geetools.batch.Export.imagecollection.toDrive to use a filename prefix 
    to define the name property when exporting images
    """
    # compat
    namePattern='{id}'
    datePattern=None
    extra=None
    dataType=datatype
    # empty tasks list
    tasklist, filenames = [], []
    # get region
    if region:
        region = tools.geometry.getRegion(region)
    # Make a list of images
    img_list = collection.toList(collection.size())
    n = 0
    while True:
        try:
            img = ee.Image(img_list.get(n))
            name = fn_prefix+"{}-{}".format(
                image.id().getInfo().split("_")[0], 
                datetime.now().strftime("%Y%d%mT%H%M%S"))
            description = utils.matchDescription(makeName(img, namePattern, datePattern, extra).getInfo())
            # convert data type
            img = utils.convertDataType(dataType)(img)
            if region is None:
                region = tools.geometry.getRegion(img)
            task = ee.batch.Export.image.toDrive(image=img,
                                                 description=description,
                                                 folder=folder,
                                                 fileNamePrefix=name,
                                                 region=region,
                                                 scale=scale,
                                                 crs=crs)
            task.start()
            if verbose: print(f"Submitted new task: taskId={task.id}, name={name}, description={description}")
            tasklist.append(task)
            filenames.append(name)
            n += 1
        except Exception as e:
            error = str(e).split(':')
            if error[0] == 'List.get': 
                if verbose: print(f"Reached end of image list at index: {n}")
                break
            else: raise e
    return tasklist, filenames

projection = ee.Projection('EPSG:4326')
geometry = ee.Geometry.Rectangle(bounds[0][0], proj=projection)
dates = (ee.Date('2017-07-01'), ee.Date('2017-07-11'))
s2_bands = ['B1','B2','B3','B4','B5','B6','B7','B8','B8A', 'B9', 'B11','B12','MSK_CLDPRB']
icol = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
       .filterBounds(geometry)
       .filterDate(*dates)
       .select(s2_bands))
print("Exporting icol with", icol.size().getInfo(), "images.")
# Geetools export
tasks, filenames = exportImageCollectionToDrive(
    collection=icol,
    region=geometry,
    folder="geeExports",
    scale=10,
    fn_prefix="498-geetoolsWithFn-",
    datatype="uint16",
    crs=projection.getInfo()["crs"],
    verbose=True
)
for t, f in zip(tasks, filenames):
    print(task, "->", f)

Exporting icol with 2 images.
Submitted new task: taskId=NMSE6VT7MIYRSLMVVI7MKFFT, name=498-geetoolsWithFn-20170526T105031-20231605T135247, description=20170702T104021_20170702T104252_T32VNN
Submitted new task: taskId=FJEIFC6GWFAQT47FXC4P3U5N, name=498-geetoolsWithFn-20170526T105031-20231605T135250, description=20170705T105031_20170705T105026_T32VNN
Reached end of image list at index: 2
<Task NAANTYZR6L36FP5QKWY6T6QA EXPORT_IMAGE: 498-test-list-13-reprojected-clipped-20170526T105031 (UNSUBMITTED)> -> 498-geetoolsWithFn-20170526T105031-20231605T135247
<Task NAANTYZR6L36FP5QKWY6T6QA EXPORT_IMAGE: 498-test-list-13-reprojected-clipped-20170526T105031 (UNSUBMITTED)> -> 498-geetoolsWithFn-20170526T105031-20231605T135250


In [133]:
tasks[0].id

'NMSE6VT7MIYRSLMVVI7MKFFT'

In [134]:
import ee
from datetime import datetime

def cast_image(image, datatype):
    try:
        return {
            "float":image.toFloat(), 
            "byte":image.toByte(), 
            "int":image.toInt(),
            "double":image.toDouble(),
            "long": image.toLong(),
            "short": image.toShort(),
            "int8": image.toInt8(),
            "int16": image.toInt16(),
            "int32": image.toInt32(),
            "int64": image.toInt64(),
            "uint8": image.toUint8(),
            "uint16": image.toUint16(),
            "uint32": image.toUint32()
        }[datatype]
    except KeyError as e:
        raise KeyError(f"Invalid datatype")

class GEEDownloader:
    def __init__(
        self,
        gee_project: str,
        crs: str="EPSG:4326",
        collection_name: str="COPERNICUS/S2_SR_HARMONIZED"
    ):
        ee.Initialize(project=gee_project)
        self.projection = ee.Projection(crs)
        self.crs = self.projection.getInfo()["crs"]
        self.crsTrasform = self.projection.getInfo()["transform"]
        self.collection_name = collection_name
    
    def downloadTimeserie(
        self,
        bbox: list, # [[xmin,ymin],[xmax,ymax]]
        fn_prefix: str, # i.e. projectId
        drive_folder: str,
        gt_date: datetime,
        date_offset_amount: int,
        date_offset_unit: str="day",
        date_offset_policy: str="both",
        bands: list=['B1','B2','B3','B4','B5','B6','B7','B8','B8A', 'B9', 'B11','B12','MSK_CLDPRB'],
        datatype: str="uint16"
    ):
        assert date_offset_policy in ["before", "after", "both"], f'Invalid date_offset_policy: {date_offset_policy}. Value must be in {["before", "after", "both"]}.'
        # Configure
        geometry = ee.Geometry.Rectangle(bbox, proj=self.projection)
        if date_offset_policy == "before": start_offset, end_offset = -date_offset_amount, 0
        elif date_offset_policy == "after": start_offset, end_offset = 0, date_offset_amount
        else: start_offset, end_offset = -date_offset_amount, date_offset_amount
        dates = (
            ee.Date(gt_date).advance(start_offset, date_offset_unit), 
            ee.Date(gt_date).advance(end_offset, date_offset_unit)
        )
        # ImageCollection
        icol = (ee.ImageCollection(self.collection_name)
           .filterBounds(geometry)
           .filterDate(*dates)
           .select(s2_bands))
        # Create download tasks
        fn_prefix = fn_prefix+"-GEE-"+self.collection_name.replace("/", "_")+"-"
        tasks, filenames = exportImageCollectionToDrive(
            collection=icol,
            region=geometry,
            folder=drive_folder,
            scale=10,
            fn_prefix=fn_prefix,
            datatype=datatype,
            crs=projection.getInfo()["crs"],
            verbose=True
        )
        return tasks, filenames
    
def download_gee_tasks(drive, tasks, filenames, drive_folder, local_folder):
    folderId = drive.getFolderId(drive_folder)
    while len(tasks)>0:
        for filename, task in zip(filenames, tasks):
            if task.state == "COMPLETED":
                try:
                    print(f"Downloading {drive_folder}/{filename}")
                    fileId = drive.getFileId(folename, folderId)
                    drive.downloadFile(fileId=fileId, localdir=local_folder)
                    try:
                        drive.deleteFile(fileId)
                        tasks.remove(task)
                        filenames.remove(filename)
                    except:
                        print(f"Could not delete file: fileId={fileId}, filename={filename}")
                        print(e)
                except:
                    print(f"Could not download task: taskId={task.id}, filename={filename}")
                    print(e)                            
            elif task.state == "FAILED": 
                tasks.remove(task)
                filenames.remove(task)
                print(f"Task(taskId={task.id}, fileName={filename}) has failed.")
            elif task.state == "CANCELLED": 
                tasks.remove(task)
                filenames.remove(task)
                print(f"Task(taskId={task.id}, fileName={filename}) was cancelled.")
            else:
                print(f"Unkown task state: {task.state}")

In [138]:
from gdrive_handler import GDriveV3Wrapper
drive = GDriveV3Wrapper()

UnboundLocalError: local variable 'creds' referenced before assignment

In [139]:
from pathlib import Path
input_path = Path("whatever/the/path/is/this/5454593453245-ggdfsd.tif")
Path(str(input_path).replace(input_path.stem, input_path.stem+"_reprojected"))

PosixPath('whatever/the/path/is/this/5454593453245-ggdfsd_reprojected.tif')

In [ ]:
with gdal.Open(self.label) as label_file:
    proj_ = label_file.GetProjectionRef()
    warp_opts = gdal.WarpOptions(
        format="GTiff",
        dstSRS=proj_,
        xRes=10.0, yRes=-10.0)
    x_ds = gdal.Warp(output_path, input_path, options=warp_opts)
    x_ds = None 